<a href="https://colab.research.google.com/github/JitheshK-11/arecanut-yield-predictor/blob/main/mainproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.base import clone

# ---------------- CONFIG ----------------
DATA_PATH = "/content/sample_data/arecanut_all_major_districts_dataset_synthetic_yield.csv"
OUTPUT_DIR = "/content/sample_data/arecanut_yield_comparison_v1"
os.makedirs(OUTPUT_DIR, exist_ok=True)
RANDOM_STATE = 42

# ---------------- 1. LOAD DATA ----------------
print("1. Loading dataset...")
df = pd.read_csv(DATA_PATH)

if "Yield_kg_per_ha" not in df.columns:
    raise ValueError("Column 'Yield_kg_per_ha' not found in dataset.")

if "Yield_kg_per_ha_synthetic" not in df.columns:
    raise ValueError("Column 'Yield_kg_per_ha_synthetic' not found. "
                     "Make sure you are using the synthetic file.")

# ---------------- 2. SELECT FEATURES ----------------
print("2. Selecting features...")

feature_cols = [
    "Area",
    "Annual_Rainfall",
    "JF",
    "MAM",
    "JJAS",
    "OND",
    "Soil_Health_Score",
    "Soil_pH",
    "Nitrogen_N_kg_ha",
    "Phosphorus_P_kg_ha",
    "Potassium_K_kg_ha"
]

missing = [c for c in feature_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required feature columns: {missing}")

X = df[feature_cols].copy()

# Targets
y_real = df["Yield_kg_per_ha"].copy()
y_synth = df["Yield_kg_per_ha_synthetic"].copy()

# Handle any NaNs (simple strategy: drop)
data = pd.concat([X, y_real, y_synth], axis=1).dropna()
X = data[feature_cols]
y_real = data["Yield_kg_per_ha"]
y_synth = data["Yield_kg_per_ha_synthetic"]

print(f"   Final usable rows: {len(X)}")

# ---------------- 3. TRAIN/TEST SPLIT ----------------
print("3. Splitting train/test...")

X_train, X_test, y_real_train, y_real_test, y_syn_train, y_syn_test = train_test_split(
    X, y_real, y_synth,
    test_size=0.2,
    random_state=RANDOM_STATE
)

# ---------------- 4. PREPROCESSOR + BASE MODEL ----------------
print("4. Building preprocessing + base model pipeline...")

num_cols = X.columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols)
    ]
)

base_rf = RandomForestRegressor(
    n_estimators=350,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

base_pipeline = Pipeline(
    steps=[
        ("pre", preprocessor),
        ("model", base_rf)
    ]
)

# ---------------- 5. HELPER FUNCTION ----------------
def train_and_evaluate(name, pipeline, X_tr, y_tr, X_te, y_te, save_prefix):
    """
    Fits the pipeline, evaluates it, saves model + plot, and returns metrics + model.
    """
    print(f"\n--- Training model for: {name} ---")
    model = clone(pipeline)
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_te)

    r2 = r2_score(y_te, y_pred)
    # Removed 'squared=False' as it might not be supported in older sklearn versions
    # Calculate RMSE manually by taking the square root of MSE
    rmse = np.sqrt(mean_squared_error(y_te, y_pred))
    mae = mean_absolute_error(y_te, y_pred)

    print(f"{name} -> R2: {r2:.4f}, RMSE: {rmse:.2f}, MAE: {mae:.2f}")

    # Save model
    model_path = os.path.join(OUTPUT_DIR, f"{save_prefix}_model.joblib")
    joblib.dump(model, model_path)

    # Plot Actual vs Predicted
    plt.figure(figsize=(6, 6))
    plt.scatter(y_te, y_pred, alpha=0.5)
    min_val = min(y_te.min(), y_pred.min())
    max_val = max(y_te.max(), y_pred.max())
    plt.plot([min_val, max_val], [min_val, max_val], "--", linewidth=2)
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    plt.title(f"{name} - Actual vs Predicted\nR2: {r2:.4f}")
    plt.tight_layout()
    fig_path = os.path.join(OUTPUT_DIR, f"{save_prefix}_actual_vs_pred.png")
    plt.savefig(fig_path)
    plt.close()

    # Feature importances from RandomForest
    rf = model.named_steps["model"]
    importances = rf.feature_importances_
    feat_imp_df = pd.DataFrame({
        "feature": num_cols,
        "importance": importances
    }).sort_values("importance", ascending=False)

    feat_imp_path = os.path.join(OUTPUT_DIR, f"{save_prefix}_feature_importances.csv")
    feat_imp_df.to_csv(feat_imp_path, index=False)

    return {
        "target": name,
        "r2": r2,
        "rmse": rmse,
        "mae": mae,
        "model_path": model_path
    }, model

# ---------------- 6. TRAIN BOTH MODELS ----------------
results = []

# Model 1: Original yield
res_real, model_real = train_and_evaluate(
    name="Original Yield (Yield_kg_per_ha)",
    pipeline=base_pipeline,
    X_tr=X_train,
    y_tr=y_real_train,
    X_te=X_test,
    y_te=y_real_test,
    save_prefix="real_yield"
)
results.append(res_real)

# Model 2: Synthetic yield
res_syn, model_syn = train_and_evaluate(
    name="Synthetic Yield (Yield_kg_per_ha_synthetic)",
    pipeline=base_pipeline,
    X_tr=X_train,
    y_tr=y_syn_train,
    X_te=X_test,
    y_te=y_syn_test,
    save_prefix="synthetic_yield"
)
results.append(res_syn)

# ---------------- 7. SAVE COMPARISON TABLE ----------------
results_df = pd.DataFrame(results)
results_path = os.path.join(OUTPUT_DIR, "yield_model_comparison.csv")
results_df.to_csv(results_path, index=False)

print("\n================= COMPARISON ================")
print(results_df)
print("=============================================")

# ---------------- 8. SELECT & SAVE FINAL MODEL ----------------
# Choose model with higher R as final
if res_syn["r2"] >= res_real["r2"]:
    best_res = res_syn
    best_model = model_syn
else:
    best_res = res_real
    best_model = model_real

final_model_path = os.path.join(OUTPUT_DIR, "final_yield_model.joblib")
joblib.dump(best_model, final_model_path)

# Save a simple text summary
summary_path = os.path.join(OUTPUT_DIR, "final_yield_model_summary.txt")
with open(summary_path, "w") as f:
    f.write("Final Selected Yield Model\n")
    f.write("==========================\n")
    f.write(f"Target: {best_res['target']}\n")
    f.write(f"R2: {best_res['r2']:.4f}\n")
    f.write(f"RMSE: {best_res['rmse']:.4f}\n")
    f.write(f"MAE: {best_res['mae']:.4f}\n")
    f.write(f"\nModel saved at: {final_model_path}\n")

print(f"\nFinal model selected for target: {best_res['target']}")
print(f"Saved as: {final_model_path}")
print(f"Summary: {summary_path}")
print(f"\nAll outputs saved in folder: {OUTPUT_DIR}")


1. Loading dataset...
2. Selecting features...
   Final usable rows: 14229
3. Splitting train/test...
4. Building preprocessing + base model pipeline...

--- Training model for: Original Yield (Yield_kg_per_ha) ---
Original Yield (Yield_kg_per_ha) -> R2: -0.0322, RMSE: 1926.24, MAE: 1399.18

--- Training model for: Synthetic Yield (Yield_kg_per_ha_synthetic) ---
Synthetic Yield (Yield_kg_per_ha_synthetic) -> R2: 0.6418, RMSE: 125.44, MAE: 99.80

================= COMPARISON ================
                                        target        r2         rmse  \
0             Original Yield (Yield_kg_per_ha) -0.032198  1926.241037   
1  Synthetic Yield (Yield_kg_per_ha_synthetic)  0.641811   125.443157   

           mae                                         model_path  
0  1399.180290  /content/sample_data/arecanut_yield_comparison...  
1    99.799103  /content/sample_data/arecanut_yield_comparison...  

Final model selected for target: Synthetic Yield (Yield_kg_per_ha_synthetic)
Sa

In [4]:
import pandas as pd
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# ---------------- CONFIG ----------------
DATA_PATH = "/content/sample_data/arecanut_all_major_districts_dataset_synthetic_yield_v2.csv"
OUTPUT_DIR = "/content/sample_data/optimized_synthetic_yield_v2_model"
os.makedirs(OUTPUT_DIR, exist_ok=True)
RANDOM_STATE = 42

# ---------------- LOAD DATA ----------------
df = pd.read_csv(DATA_PATH)

feature_cols = [
    "Area", "Annual_Rainfall", "JF", "MAM", "JJAS", "OND",
    "Soil_Health_Score", "Soil_pH",
    "Nitrogen_N_kg_ha", "Phosphorus_P_kg_ha", "Potassium_K_kg_ha"
]

X = df[feature_cols]
y = df["Yield_kg_per_ha_synthetic_v2"]

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

# Preprocessor
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), feature_cols)
])

# Base model
rf = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1)

pipeline = Pipeline([
    ("pre", preprocessor),
    ("model", rf)
])

# Random Search grid (for higher R²)
param_grid = {
    'model__n_estimators': [300, 400, 500, 700],
    'model__max_depth': [20, 30, 40, None],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4],
    'model__max_features': ['sqrt', 'log2']
}

rs = RandomizedSearchCV(
    pipeline,
    param_grid,
    n_iter=20,
    cv=3,
    scoring='r2',
    verbose=2,
    n_jobs=-1,
    random_state=42
)

print("⏳ Training optimized model on synthetic_v2 target...")
rs.fit(X_train, y_train)
best = rs.best_estimator_

# Predict
y_pred = best.predict(X_test)

r2 = r2_score(y_test, y_pred)
# Fix: Calculate RMSE by taking the square root of MSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print("\n===== Optimized Synthetic v2 Model =====")
print("Best R²:", r2)
print("RMSE:", rmse)
print("MAE:", mae)
print("Best params:", rs.best_params_)

# Save final optimized model
model_path = os.path.join(OUTPUT_DIR, "optimized_synthetic_yield_v2_model.joblib")
joblib.dump(best, model_path)
print(f"\n✅ Saved optimized model to: {model_path}")

# Plot Actual vs Predicted
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
min_val = min(y_test.min(), y_pred.min())
max_val = max(y_test.max(), y_pred.max())
plt.plot([min_val, max_val], [min_val, max_val], "--")
plt.xlabel("Actual Synthetic Yield v2 (kg/ha)")
plt.ylabel("Predicted Synthetic Yield v2 (kg/ha)")
plt.title(f"Optimized Synthetic Yield v2\nR² = {r2:.4f}")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "actual_vs_pred_synth_v2.png"))
plt.close()


⏳ Training optimized model on synthetic_v2 target...
Fitting 3 folds for each of 20 candidates, totalling 60 fits

===== Optimized Synthetic v2 Model =====
Best R²: 0.832278103752471
RMSE: 84.78980831572582
MAE: 67.44129814465282
Best params: {'model__n_estimators': 400, 'model__min_samples_split': 10, 'model__min_samples_leaf': 4, 'model__max_features': 'sqrt', 'model__max_depth': 20}

✅ Saved optimized model to: /content/sample_data/optimized_synthetic_yield_v2_model/optimized_synthetic_yield_v2_model.joblib


In [5]:
import joblib
import pandas as pd

# Load your optimized model
model_path = "/content/sample_data/optimized_synthetic_yield_v2_model/optimized_synthetic_yield_v2_model.joblib"
model = joblib.load(model_path)

# ---- SAMPLE INPUT ----
# Change the values to test different scenarios
sample = pd.DataFrame([{
    "Area": 5200,
    "Annual_Rainfall": 3450,
    "JF": 18,
    "MAM": 240,
    "JJAS": 3000,
    "OND": 180,
    "Soil_Health_Score": 3.8,
    "Soil_pH": 6.4,
    "Nitrogen_N_kg_ha": 75,
    "Phosphorus_P_kg_ha": 22,
    "Potassium_K_kg_ha": 210
}])

# Predict the synthetic yield v2
predicted_yield = model.predict(sample)[0]
print(f"Predicted Synthetic Yield v2: {predicted_yield:.2f} kg/ha")

# If you want predicted production (tons):
predicted_production = predicted_yield * sample["Area"].iloc[0] / 1000
print(f"Predicted Production: {predicted_production:.2f} tons")


Predicted Synthetic Yield v2: 2714.97 kg/ha
Predicted Production: 14117.83 tons
